In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
import warnings
warnings.filterwarnings("ignore")

## Data pre-processing

In [3]:
# Import data, remove na and duplicate rows 
df = pd.read_csv('creditcard.csv')
df.dropna()
df.drop_duplicates()
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
# Convert Time variable to hour
df['Hour'] = df['Time'].apply(datetime.fromtimestamp).dt.hour
df.drop(['Time'], axis=1, inplace = True)

In [5]:
X = df.drop(['Class'], axis = 1)
y = df['Class']

## Machine Learning Pipeline

Choose metric for evaluating models

In [6]:
from sklearn.metrics import fbeta_score, make_scorer, confusion_matrix, ConfusionMatrixDisplay

f2_scorer = make_scorer(fbeta_score, beta = 2)

def metric(y_test, y_pred):
    print('F2 Score: %.4f' % fbeta_score(y_test, y_pred, beta = 2))
    cm = confusion_matrix(y_test, y_pred, labels = [0, 1])
    disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = ['Legitimate','Fraud'])
    return disp

Define standard scaler

In [7]:
from sklearn.preprocessing import StandardScaler

std_slc = StandardScaler()

### Find the best model

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=68)

#### Logistic Regression

In [9]:
from sklearn.linear_model import LogisticRegression

# Define models and parameters
model = LogisticRegression(random_state = 68)
c_values = [100, 10, 1.0, 0.1, 0.01]

# Define pipeline and grid search
pipeline = Pipeline(steps=[('scaler', std_slc),('model', model)])
grid = dict(model__C = c_values)
grid_search = GridSearchCV(estimator = pipeline, param_grid = grid, cv = cv, scoring = f2_scorer)

# Find the best hyperparameters
grid_result = grid_search.fit(X,y)

# Summarize results
print("Best: %f using %s" %(grid_result.best_score_, grid_result.best_params_))

Best: 0.657238 using {'model__C': 1.0}


#### Decision Tree

In [10]:
from sklearn.tree import DecisionTreeClassifier

# Define models and parameters
model = DecisionTreeClassifier(random_state=68) 
max_depth = [2,3,4,5,6,7,8,9,10,20]

# Define pipeline and grid search
pipeline = Pipeline(steps=[('scaler', std_slc),
                           ('model', model)])
grid = dict(model__max_depth=max_depth)
grid_search = GridSearchCV(estimator=pipeline, param_grid=grid, cv=cv,scoring=f2_scorer)

# Find best hyperparameters
grid_result = grid_search.fit(X, y)

# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.804354 using {'model__max_depth': 6}


#### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
    
# Define models and parameters
model = RandomForestClassifier(random_state=68)
max_features = ['sqrt', 'log2']
n_estimators = [25,50,100]

# Define pipeline and grid search
pipeline = Pipeline(steps = [('scaler', std_slc),
                           ('model', model)])
grid = dict(model__max_features = max_features, model__n_estimators = n_estimators)
grid_search = GridSearchCV(estimator = pipeline, param_grid = grid, cv = cv,scoring = f2_scorer)

# Find best hyperparameters
grid_result = grid_search.fit(X, y)

# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_)) 

### Evaluate the best model as result

In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size = 0.20, random_state = 68, stratify = df[['Class']])

In [ ]:
# Train set
X_train = df_train.drop('Class', axis = 1)
y_train = df_train['Class']

print("Fraudulent transactions are %.2f%% of the train set." % (y_train.value_counts()[1] * 100 / len(y_train)))

# Test set
X_test = df_test.drop('Class', axis = 1)
y_test = df_test['Class']

In [ ]:
 # Amout variable
scaler_Amount = StandardScaler().fit(X_train['Amount'].values.reshape(-1, 1))
X_train['Amount'] = scaler_Amount.transform(X_train['Amount'].values.reshape(-1, 1))
X_test['Amount'] = scaler_Amount.transform(X_test['Amount'].values.reshape(-1,1))
# Hour variable
scaler_Hour = StandardScaler().fit(X_train['Hour'].values.reshape(-1, 1))
X_train['Hour'] = scaler_Hour.transform(X_train['Hour'].values.reshape(-1, 1))
X_test['Hour'] = scaler_Hour.transform(X_test['Hour'].values.reshape(-1, 1))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(max_features='log2', n_estimators=25,␣ 􏰁→random_state=68)
y_pred = model.fit(X_train, y_train).predict(X_test)
fig = metric(y_test, y_pred)

In [ ]:
fig.plot()
plt.savefig('plots/confusion_matrix.png')